In [1]:
# Chạy cell này đầu tiên
from bs4 import BeautifulSoup
from IPython.display import clear_output
import requests
import json
import os
import pandas as pd

In [2]:
# Merge file
# Không cần nhạy cell này
'''
from unidecode import unidecode

json_files = {
    "VN": "artist_VietNam.json", 
    "JP": "artist_Japan.json", 
    "EU": "artist_EU.json",
    "USUK": "artist_USUK.json",
    "KR": "artist_Korea.json",
    "Latin": "artist_Latin.json"
}


def normalize_name(name):
    return unidecode(name.lower())

def filter_similar_names(names):
    unique_names = set()
    filtered_names = []
    for name in names:
        normalized_name = normalize_name(name)
        if normalized_name not in unique_names:
            unique_names.add(normalized_name)
            filtered_names.append(name)
    return filtered_names

# Initialize an empty dictionary to store names and areas
name_area_dict = {}

# Loop through each JSON file
for area, json_file in json_files.items():
    with open(f"./{json_file}", 'r') as file:
        names = json.load(file)
        filtered_names = filter_similar_names(names)
        for name in filtered_names:
            if name in name_area_dict:
                name_area_dict[name] += f';{area}'
            else:
                name_area_dict[name] = area

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(name_area_dict.items()), columns=['Name', 'Area'])

# Save the DataFrame to a CSV file
df.sort_values(by=['Name']).to_csv('artistDataset.csv', index=False)
print("CSV file created successfully.")
'''
''''''

''

In [3]:
# Sau đó chạy cell này
def getSongInfo(name, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        links = soup.select(".play-this-track-playlinks .play-this-track-playlink")
        result = {}
        for link in links:
            result[link.getText().strip()] = link['href']
        return result
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}
def crawlAllSongOfArtist(artist):
    try:
        allSongsList = []
        artistQuery = artist.replace("+", "%252B").replace(" ", "+")
        url = f"https://www.last.fm/music/{artistQuery}/+tracks?date_preset=ALL"
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        pages = soup.find_all(attrs={"class": "pagination-page"})
        last_page = int(pages[-1].find('a').get_text().strip()) if len(pages) else 1
        for i in range(1, last_page+1):
        # For test using: 
        #for i in range(1, 2):
            res = requests.get(f"{url}&page={i}")
            res.raise_for_status()
            pg = res.text
            soup = BeautifulSoup(pg, 'html.parser')
            songs = soup.select("tr.chartlist-row > .chartlist-name > a")
            for song in songs:
            # For test using: 
            #for song in songs[:5]:
                songName = song.getText()
                songUrl = song['href']
                allSongsList.append([songName, getSongInfo(songName, f"https://www.last.fm{songUrl}")])
                print(f"artist: {artist} - page: {i}/{last_page} - song: {songName}")
                clear_output(wait=True)
        return allSongsList
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [4]:
'''
Bắt đầu từ đây 
1. Đổi lại firstIndex và lastIndex dựa vào danh sách nghệ sĩ được yêu cầu
Minh: 0 - 6320
Ngọc: 6321 - 12640
Quân: 12641 - 18960
Giang: 18961 - 25280
Long: 25280 - 31605
2. Chạy cell này. Giữ internet ổn định và cắm máy đấy rồi đi làm việc khác
Lưu ý: * Sau mỗi nghệ sĩ crawl thành công sẽ lưu vào file. 
Do đó có thể chạy tiếp cell này sau khi stop, tiến trình sẽ tiếp tục.
Tuy nhiên nên hạn chế để tránh lỗi

* Nếu thấy log bên dưới error, lập tức bấm Interrupt và kiểm tra lại internet
'''
####
firstIndex = 7701
lastIndex = 7900
####
target_file_data = None  # Initialize as a list
targetFileName = f'./songData{firstIndex}-{lastIndex}.csv'

# File danh sách nghệ sĩ
artist_df = pd.read_csv('artistDataset.csv')

if os.path.exists(targetFileName):
    target_file_data = pd.read_csv(targetFileName)
else:
    column_names = [
        "title", 
        "artist", 
        "area", 
        "youtube_link", 
        "spotify_link", 
        "release_date", 
        "youtube_view_count", 
        "youtube_like_count", 
        "tag",
        "duration",
        "popularity",
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "loudness",
        "tempo",
    ]
    target_file_data = pd.DataFrame(columns=column_names)
    target_file_data.to_csv(targetFileName, index=False)
    target_file_data = pd.read_csv(targetFileName)

for i in range(firstIndex, lastIndex + 1):
    dfs_to_concat = []
    artist = artist_df.loc[i, "Name"]
    if artist not in target_file_data["artist"].unique():
        for song_info in crawlAllSongOfArtist(artist):
            yt_view_count = None
            yt_like_count = None
            '''
            popularity = None
            duration = None
            danceability = None
            energy = None
            speechiness = None
            acousticness = None
            instrumentalness = None
            liveness = None
            valence = None
            loudness = 0
            tempo = None
            release_date = None
            '''
            if "YouTube" in song_info[1]:
                yt_API_res = requests.get(f'https://api.socialcounts.org/youtube-video-live-view-count/{song_info[1]["YouTube"].split("?v=")[-1]}').json()
                yt_view_count = yt_API_res["est_sub"] if "est_sub" in  yt_API_res else None
                yt_like_count = yt_API_res["table"][0]["count"] if "table" in  yt_API_res else None
            '''
            if "Spotify" in song_info[1]:
                spotify_id = song_info[1]["Spotify"].split("/")[-1]
                spotify_feature_API_res = requests.get(f'https://www.chosic.com/api/tools/audio-features/{spotify_id}')
                spotify_info_API_res = requests.get(f'https://www.chosic.com/api/tools/tracks/{spotify_id}')
                popularity = spotify_info_API_res["popularity"] if "popularity" in spotify_info_API_res else None
                release_date = spotify_info_API_res["album"]["release_date"] if "album" in spotify_info_API_res else None
                duration = spotify_info_API_res["duration_ms"] if "duration_ms" in spotify_info_API_res else None
                danceability = spotify_info_API_res["danceability"] if "danceability" in spotify_info_API_res else None
                energy = spotify_info_API_res["energy"] if "energy" in spotify_info_API_res else None
                speechiness = spotify_info_API_res["speechiness"] if "speechiness" in spotify_info_API_res else None
                acousticness = spotify_info_API_res["acousticness"] if "acousticness" in spotify_info_API_res else None
                instrumentalness = spotify_info_API_res["instrumentalness"] if "instrumentalness" in spotify_info_API_res else None
                liveness = spotify_info_API_res["liveness"] if "liveness" in spotify_info_API_res else None
                valence = spotify_info_API_res["valence"] if "valence" in spotify_info_API_res else None
                loudness = spotify_info_API_res["loudness"] if "loudness" in spotify_info_API_res else None
                tempo = spotify_info_API_res["tempo"] if "tempo" in spotify_info_API_res else None
            '''
            df_to_append = pd.DataFrame({
                "title": [song_info[0]],
                "artist": [artist],
                "area": [artist_df.loc[i, "Area"]],
                "youtube_link": [song_info[1]["YouTube"] if "YouTube" in song_info[1] else None],
                "spotify_link": [song_info[1]["Spotify"] if "Spotify" in song_info[1] else None],
                "youtube_view_count": [yt_view_count],
                "youtube_like_count": [yt_like_count],
                #"duration": [duration],
                #"danceability": [danceability],
                #"energy": [energy],
                #"speechiness": [speechiness],
                #"acousticness": [acousticness],
                #"instrumentalness": [instrumentalness],
                #"liveness": [liveness],
                #"valence": [valence],
                #"loudness": [loudness],
                #"tempo": [tempo],
                "tag": None
            }) 
            dfs_to_concat.append(df_to_append)
        target_file_data = pd.concat([target_file_data] + dfs_to_concat, ignore_index=True)
        target_file_data.to_csv(targetFileName, index=False)

Request error: HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /music/Gloria+Estefan/+tracks?date_preset=ALL&page=7 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019955581C30>: Failed to resolve 'www.last.fm' ([Errno 11001] getaddrinfo failed)"))
Request error: HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /music/Gloria+Gaynor/+tracks?date_preset=ALL (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019955582590>: Failed to resolve 'www.last.fm' ([Errno 11001] getaddrinfo failed)"))
Request error: HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /music/Gloria+Trevi/+tracks?date_preset=ALL (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019955583400>: Failed to resolve 'www.last.fm' ([Errno 11001] getaddrinfo failed)"))
Request error: HTTPSConnectionPool(host='www.last.fm', port=

https://api.socialcounts.org/youtube-video-live-view-count/8xg3vE8Ie_E